In this notebook, the data used to produce the key figures in the results section can be generated. In the file Main_document.ipynb, the data is used to create the figures. First the necessary libraries are imported. Problem formulation id 2 is mostly used in the generation of data.

In [ ]:
# Import necessary libraries
import pandas as pd 
import os
from datetime import datetime
import numpy as np
from ema_workbench import (
    Model,
    Policy,
    Scenario,
    ema_logging,
    SequentialEvaluator,
    MultiprocessingEvaluator,
    Samplers,
)
from problem_formulation import get_model_for_problem_formulation, sum_over, sum_over_time
dike_model, planning_steps = get_model_for_problem_formulation(2)


# Display all columns in the DataFrame
pd.set_option('display.max_columns', None)

Firstly, we define a function allowing us to save the results of a run, with a name and a set folder.

In [ ]:
def save_experiments_and_outcomes(name, experiments, outcomes, folder_name="sobol_results"):
    """
    Saves the experiments and outcomes to CSV files in the specified folder,
    with filenames that include a custom name and timestamp.

    Parameters:
    - name (str): Custom label for the output files (e.g., 'policy6_seed20').
    - experiments (pd.DataFrame): The experiments DataFrame.
    - outcomes (dict): The outcomes dictionary from the EMA Workbench.
    - folder_name (str): The name of the folder where files will be saved. Default is "results".
    """
    # Get timestamp
    current_date = datetime.now().strftime('%Y-%m-%d')

    # Ensure folder exists
    os.makedirs(folder_name, exist_ok=True)

    # Define file paths
    exp_path = os.path.join(folder_name, f'{name}_experiments_{current_date}.csv')
    out_path = os.path.join(folder_name, f'{name}_policies_{current_date}.csv')

    # Save experiments
    experiments.to_csv(exp_path, index=False)

    # Flatten outcomes and save
    flattened_data = {key: value.flatten() for key, value in outcomes.items()}
    df_outcomes = pd.DataFrame(flattened_data)
    df_outcomes.to_csv(out_path, index=False)

    print(f"Saved experiments and outcomes as '{name}' in '{folder_name}' (timestamp: {current_date})")


## Open Exploration

### 'Zero policy'
In this section of the notebook, the data required for the subsection 'Zero Policy' are generated.

## Hier doe jij dingen PIEN!!!

### Subspace partitioning
In this section of the notebook, the data required for the subsection Subspace partitioning are generated. The Latin hypercube sample consists of 400 scenarios and 600 policies.

In [ ]:
n_scenarios = 600
n_policies = 400

In [ ]:
# running the model to get the LHS data for the subspace partitioning
np.random.seed(21)
with MultiprocessingEvaluator(dike_model) as evaluator:
    results = evaluator.perform_experiments(scenarios=n_scenarios, policies=n_policies, uncertainty_sampling=Samplers.LHS, lever_sampling=Samplers.LHS)
experiments, outcomes = results
save_experiments_and_outcomes('LHS_using_pf2', experiments, outcomes, folder_name="results")

## Sensitivity analysis (Sobol Indices)
In this section of the notebook, the data required for the subsection Sobol Indices are generated. The zero-policy and an average scenario are defined and then used to perform experiments.

In [ ]:
# Create a dictionary with all levers set to 0
def get_do_nothing_dict():
    return {l.name: 0 for l in dike_model.levers}

# Create a policy using the do-nothing dictionary
policies = [
    Policy(
        "policy 0",
        **dict(
            get_do_nothing_dict()
        )
    ),]

In [ ]:
# This dataframe is used to show the average values of the uncertainty parameters
df = pd.read_csv("results/scenario_space_100000_experiments_2025-06-11_21-41.csv")
df.describe()

In [ ]:
# Create a scenario with average values for the uncertainties
reference_values = {
    "Bmax": 190,
    "Brate": 4.167,
    "pfail": 0.5,
    "discount rate 0": 3,
    "discount rate 1": 3,
    "discount rate 2": 3,
    "ID flood wave shape": 4, # Arbritary value, there is no average value for this parameter
}

# Create a scenario dictionary and then a Scenario object
scenario_dict = {}
for key in dike_model.uncertainties:
    name_split = key.name.split("_")
    if len(name_split) == 1:
        scenario_dict[key.name] = reference_values.get(key.name)
    else:
        scenario_dict[key.name] = reference_values.get(name_split[1])
scenario = Scenario("default", **scenario_dict)
scenarios = [scenario,]


Below the experiments can be run, the number of scenarios and policies chosen is very high, as this is needed to perform a sensitivity analysis using Sobol Indices.

In [ ]:
# The amount of scenarios and policies must be a power of two for the Sobol' sequence
n_scenarios = 4096
n_policies = 4096

In [ ]:
# running the model to get the sobol indices for the scenario space
np.random.seed(21)
with MultiprocessingEvaluator(dike_model) as evaluator:
    results = evaluator.perform_experiments(scenarios=n_scenarios, policies=policies, uncertainty_sampling=Samplers.SOBOL)
experiments, outcomes = results
# Save the experiments and outcomes to CSV files
save_experiments_and_outcomes('SOBOL_scenarios', experiments, outcomes, folder_name="sobol_results")

In [ ]:
# running the model to get the sobol indices for the policy space
np.random.seed(21)
with MultiprocessingEvaluator(dike_model) as evaluator:
    results = evaluator.perform_experiments(scenarios=scenarios, policies=n_policies, lever_sampling=Samplers.SOBOL)   
experiments, outcomes = results
# Save the experiments and outcomes to CSV files
save_experiments_and_outcomes('SOBOL_policies', experiments, outcomes, folder_name="sobol_results")